In [75]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('notebook')
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC, LinearSVR
#from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
#import lightgbm as lgb

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import make_scorer, f1_score, fbeta_score, roc_auc_score, auc, roc_curve, precision_score, recall_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneOut
from sklearn.neighbors import DistanceMetric
from scipy import stats
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
def split_train_test_data (data, id_column, name_target, test_size, random_state):
    
    X_all = data.drop([name_target], axis = 1)
    y_all = data[name_target]

    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = test_size ,random_state = random_state)

    print ("Training set has {} samples and {} features.".format(X_train.shape[0], X_train.shape[1]))
    print ("Testing set has {} samples and {} features.".format(X_test.shape[0], X_test.shape[1]))  
    
    X_train.reset_index(drop=True, inplace = True)
    X_test.reset_index(drop=True, inplace = True)
    y_train.reset_index(drop=True, inplace = True)
    y_test.reset_index(drop=True, inplace = True)
    
    id_train = X_train[id_column]
    id_test = X_test[id_column]
    #print(id_train, id_test)
    X_train = X_train.drop([id_column], axis = 1)
    X_test = X_test.drop([id_column], axis = 1)
       
    
    return X_train, X_test, y_train, y_test, id_train, id_test



def fix_columns_data_scoring (new_data, train_columns):
    missing_cols = set(train_columns) - set(new_data.columns)
    for c in missing_cols:
        new_data[c] = 0
        
    assert(set(train_columns) - set(new_data.columns) == set())
    extra_cols = set(new_data.columns) - set(train_columns)
    if extra_cols:
        print('columns extra:', extra_cols)
    new_data = new_data[train_columns]
    return new_data

In [27]:
data = pd.read_csv('data_examples/data_classif.csv', sep= ';')

data = data.reset_index()
data.rename(columns = {'index': 'id_'}, inplace=True)
data.head()


,id_,y,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,0,-0.560476,-0.995799,-0.511604,-0.150307,0.196550,1,0,0,0
1,2,0,-0.230177,-1.039955,0.236938,-0.327757,0.650113,1,0,0,0
2,3,0,1.558708,-0.017980,-0.541589,-1.448165,0.671004,0,0,1,0
3,4,0,0.070508,-0.132175,1.219228,-0.697285,-1.284158,1,1,1,0
4,5,0,0.129288,-2.549343,0.174136,2.598490,-2.026110,1,0,0,1


In [34]:
# 1. split na base
X_train, X_test, y_train, y_test, id_train, id_test = split_train_test_data(data, 'id_', 'y', 0.2,1)

Training set has 800 samples and 10 features.
Testing set has 200 samples and 10 features.


In [ ]:
# 2. teste de modelos

entradas

algoritmos para testar
pipeline algoritmos



In [ ]:
# param_svm = [{'clf__C': [1, 10, 100, 1000]}
# param_gb = [{'clf__C': [1, 10, 100, 1000]}
# param_rf = [{'clf__C': [1, 10, 100, 1000]}

In [63]:
from sklearn.pipeline import Pipeline

In [62]:
estimators = [ RandomForestClassifier(),  LogisticRegression()]
params = [{'clf__C': [1, 10, 100, 1000]}, 
          {'clf__n_estimators': [1, 10, 100, 1000]},
            {'clf__n_estimators': [1, 10, 100, 1000]}]

In [70]:
sorted(pipe.get_params().keys())

NameError: name 'pipe' is not defined

In [76]:
#pipe = Pipeline(['clf',RandomForestClassifier()])     
pipeline = Pipeline([
     ('tfidf', TfidfVectorizer()),
     ('classifier', RandomForestClassifier())
])

In [71]:
a, b = grid_search_model(X_train, y_train, pipe,  {'clf__n_estimators': [1, 10, 100, 1000]}, 'roc_auc', KFold(5), 3)
    

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

In [66]:
for i in estimators:
    pipe = Pipeline(['clf', estimators[i]])           
    a, b = grid_search_model(X_train, y_train, pipe, params, 'roc_auc', KFold(5), 3)
    

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

In [ ]:
GridSearchCV()

In [43]:
def grid_search_model (x, y, clf, parameters, scorer, cv, verbose):
    grid_obj = GridSearchCV(clf, parameters, scoring = scorer, cv =  cv, verbose= verbose)
    start = time()
    grid_fit = grid_obj.fit(x, y)
    end = time()
    print ("Best score: {}".format(grid_fit.best_score_))
    print ("Best parameters: {}".format(grid_fit.best_params_))
    print ("Find best parameterers in {:.4f} seconds.".format(end - start))
    return grid_fit, grid_fit.best_estimator_


In [45]:
params = {
    'n_estimators' :[10]
}

a,b = grid_search_model(X_train, y_train, RandomForestClassifier(), params, 'roc_auc',cv=KFold(5), verbose =3 )

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] n_estimators=10 .................................................
[CV] ........ n_estimators=10, score=0.9999999999999999, total=   0.0s
[CV] n_estimators=10 .................................................
[CV] ....................... n_estimators=10, score=1.0, total=   0.0s
[CV] n_estimators=10 .................................................
[CV] ....................... n_estimators=10, score=1.0, total=   0.0s
[CV] n_estimators=10 .................................................
[CV] ....................... n_estimators=10, score=1.0, total=   0.0s
[CV] n_estimators=10 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ....................... n_estimators=10, score=1.0, total=   0.0s
Best score: 1.0
Best parameters: {'n_estimators': 10}
Find best parameterers in 0.2750 seconds.


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [57]:
a.cv_results_['mean_test_score']

array([1.])